Current Work in progress test2

In [16]:
import pandas as pd
import numpy as np
import folium
import json, requests

In [17]:
#Foursquare API Login
foursquare_id_client = 'PGHLXB52TSXRO5T5IIWLEG3HKZLZXLWOOX0HR54MENTTZC4M'
foursquare_id_secret = 'S2WHF2IA1EQLEATW5UMIZWOR0JUIMUZL4TJ4ATHHXS1EHATI'
foursquare_id_version = '20200801'

In [52]:
def foursquare_url(endpoint, **kwargs):
    # Foursquare Login Credentials
    foursquare_id_client = 'PGHLXB52TSXRO5T5IIWLEG3HKZLZXLWOOX0HR54MENTTZC4M'
    foursquare_id_secret = 'S2WHF2IA1EQLEATW5UMIZWOR0JUIMUZL4TJ4ATHHXS1EHATI'
    foursquare_id_version = '20200801'
    
    # Lists of allowed keywords
    search_keywords_simple = ['radius', 'query', 'limit']
    near_flag = False
    coordinates_flag = False
    
    
    
    # Construct base url with selected api endpoint and authentication credentials
    url = 'https://api.foursquare.com/v2/venues/'
    url = url + '{}?&client_id={}&client_secret={}&={}'.format(endpoint, foursquare_id_client, 
                                                               foursquare_id_secret, foursquare_id_version)
    if endpoint == 'search':
        for key, value in kwargs.items():
            
            if key in search_keywords_simple:
                url = url + '&{}={}'.format(key, value)
            
            elif (str(key) == 'lattitude' or str(key) == 'longitude') and (not coordinates_flag):
                if not near_flag:
                    try:
                        l1 = kwargs.get(lattitude)
                        l2 = kwargs.get(longitude)
                        url = url + '&ll={},{}'.format(l1, l2)
                    except:
                        print('Error with entering coordinates')
                        return               
                
            else:
                print("Wrong key: %s" % key)
                return
             
    return url

In [53]:
url = foursquare_url('search', radius = '500', lattitude = '50.0782184', longitude = '8.2397608')
url

Error with entering coordinates


In [48]:
lattitude = '50.0782184'
longitude = '8.2397608'
endpoint = 'search'
radius = 3000
limit = 100
query = 'Palast'

In [49]:
url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    endpoint, foursquare_id_client, foursquare_id_secret, foursquare_id_version, 
    lattitude, longitude, radius, limit, query
    )
url

'https://api.foursquare.com/v2/venues/search?&client_id=PGHLXB52TSXRO5T5IIWLEG3HKZLZXLWOOX0HR54MENTTZC4M&client_secret=S2WHF2IA1EQLEATW5UMIZWOR0JUIMUZL4TJ4ATHHXS1EHATI&v=20200801&ll=50.0782184,8.2397608&radius=3000&limit=100&query=Palast'

In [50]:
results = requests.get(url).json()